In [1]:
## 02-Regression-Homework

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
### Dataset
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'

data = pd.read_csv(url)
data.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [4]:
### Preparing the dataset 

cols = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year','fuel_efficiency_mpg']
df_col = data[cols]
df_col

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,170,159.0,3413.433759,2003,13.231729
1,130,97.0,3149.664934,2007,13.688217
2,170,78.0,3079.038997,2018,14.246341
3,220,NaN,2542.392402,2009,16.912736
4,210,140.0,3460.870990,2009,12.488369
...,...,...,...,...,...
9699,140,164.0,2981.107371,2013,15.101802
9700,180,154.0,2439.525729,2004,17.962326
9701,220,138.0,2583.471318,2008,17.186587
9702,230,177.0,2905.527390,2011,15.331551


In [5]:
### EDA: long tail
mpg = df_col['fuel_efficiency_mpg']
print('skewness =', mpg.skew())
print('describe =', mpg.describe())
sns.histplot(df_col.fuel_efficiency_mpg, bins=50)

skewness = -0.012062219273507929
describe = count    9704.000000
mean       14.985243
std         2.556468
min         6.200971
25%        13.267459
50%        15.006037
75%        16.707965
max        25.967222
Name: fuel_efficiency_mpg, dtype: float64


<Axes: xlabel='fuel_efficiency_mpg', ylabel='Count'>

In [6]:
sns.histplot(mpg)


<Axes: xlabel='fuel_efficiency_mpg', ylabel='Count'>

In [7]:
"""
There's one column with missing values. What is it?

* `'engine_displacement'`
* `'horsepower'`
* `'vehicle_weight'`
* `'model_year'`
"""
# Q1: Which column has missing values?
df_col.isnull().sum()


engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

In [8]:
"""
What's the median (50% percentile) for variable `'horsepower'`?

- 49
- 99
- 149
- 199
"""
df_col['horsepower'].describe()
# Q2: median horsepower
median_hp = df_col['horsepower'].median()
print("Median hp:", median_hp)


Median hp: 149.0


In [9]:
### Prepare and split the dataset
"""
* Shuffle the dataset (the filtered one you created above), use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
"""
n = len(df_col)

n_test = int(n*0.2)
n_val = int(n*0.2)
n_train = n - n_test - n_val

In [10]:
n,n_val + n_test + n_train

(9704, 9704)

In [11]:
# shuffle

idx = np.arange(n)
idx

array([   0,    1,    2, ..., 9701, 9702, 9703])

In [12]:
#seed

np.random.seed(42)
np.random.shuffle(idx)
idx

array([ 483, 7506, 8795, ..., 5390,  860, 7270])

In [13]:
df_train = df_col.iloc[idx[:n_train]]
df_val = df_col.iloc[idx[n_train: n_train + n_val]]
df_test = df_col.iloc[idx[n_train+n_val:]]
len(df_col), len(df_train) + len(df_val) + len(df_test)

(9704, 9704)

In [14]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [15]:
df_val

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,180,112.0,2772.134410,2020,15.366341
1,260,138.0,3014.061530,2022,14.601182
2,140,181.0,3652.186151,2016,12.247032
3,270,105.0,2753.040847,2023,15.136400
4,170,224.0,4163.375137,2003,10.340419
...,...,...,...,...,...
1935,290,205.0,3403.401496,2019,13.868607
1936,200,159.0,2752.516039,2009,15.665403
1937,230,202.0,3303.000688,2008,13.613127
1938,220,169.0,3293.420251,2008,13.733343


In [16]:
y_train = df_train.fuel_efficiency_mpg
y_val = df_val.fuel_efficiency_mpg
y_test = df_test.fuel_efficiency_mpg

In [17]:
y_val

0       15.366341
1       14.601182
2       12.247032
3       15.136400
4       10.340419
          ...    
1935    13.868607
1936    15.665403
1937    13.613127
1938    13.733343
1939    15.816050
Name: fuel_efficiency_mpg, Length: 1940, dtype: float64

In [18]:
del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

In [19]:
### Question 3
"""
* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

Options:

- With 0
- With mean -
- Both are equally good
"""

'\n* We need to deal with missing values for the column from Q1.\n* We have two options: fill it with 0 or with the mean of this variable.\n* Try both options. For each, train a linear regression model without regularization using the code from the lessons.\n* For computing the mean, use the training only!\n* Use the validation dataset to evaluate the models and compare the RMSE of each option.\n* Round the RMSE scores to 2 decimal digits using `round(score, 2)`\n* Which option gives better RMSE?\n\nOptions:\n\n- With 0\n- With mean -\n- Both are equally good\n'

In [20]:
# Q3.1: missing value
df_train_0 = df_train.copy()

df_train_0.columns
df_train_0.isnull().sum()

engine_displacement      0
horsepower             429
vehicle_weight           0
model_year               0
dtype: int64

In [21]:
# Q3.2: filling nan value with 0
df_train_0.fillna(0).isnull().sum()

engine_displacement    0
horsepower             0
vehicle_weight         0
model_year             0
dtype: int64

In [22]:
# Q3.3: training
X_train = df_train_0.fillna(0).values
X_train

array([[ 220.        ,  144.        , 2535.88759124, 2009.        ],
       [ 160.        ,  141.        , 2741.17048439, 2019.        ],
       [ 230.        ,  155.        , 2471.88023726, 2017.        ],
       ...,
       [ 210.        ,  152.        , 2500.17568746, 2020.        ],
       [ 250.        ,  154.        , 2254.31324473, 2002.        ],
       [ 210.        ,  152.        , 2435.20360628, 2004.        ]])

In [23]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X) # gram matrix
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)


    return w_full[0], w_full[1:]

In [24]:
def rmse(y, y_pred):
    se = (y - y_pred)**2
    mse = se.mean()
    return np.sqrt(mse)

In [25]:
w0, w = train_linear_regression(X_train, y_train)

In [26]:
y_pred = w0 + X_train.dot(w)
y_pred

array([17.33526223, 16.29296317, 17.69885099, ..., 17.54541307,
       18.78287539, 17.86564843])

In [27]:
score_0 = round(rmse(y_train, y_pred), 2)
score_0

0.52

In [28]:
#Q3.2: filling nan value with mean
df_train_mean = df_train.copy()

df_train_mean.columns
df_train_mean.isnull().sum()

engine_displacement      0
horsepower             429
vehicle_weight           0
model_year               0
dtype: int64

In [29]:
df_train_mean['horsepower'] = df_train_mean['horsepower'].fillna(df_train_mean['horsepower'].mean())
df_train_mean.isnull().sum()

engine_displacement    0
horsepower             0
vehicle_weight         0
model_year             0
dtype: int64

In [30]:
X_train_mean = df_train_mean.values
X_train_mean

array([[ 220.        ,  144.        , 2535.88759124, 2009.        ],
       [ 160.        ,  141.        , 2741.17048439, 2019.        ],
       [ 230.        ,  155.        , 2471.88023726, 2017.        ],
       ...,
       [ 210.        ,  152.        , 2500.17568746, 2020.        ],
       [ 250.        ,  154.        , 2254.31324473, 2002.        ],
       [ 210.        ,  152.        , 2435.20360628, 2004.        ]])

In [31]:
w0, w = train_linear_regression(X_train_mean, y_train)

In [32]:
y_pred_mean = w0 + X_train_mean.dot(w)
y_pred_mean

array([17.25910991, 16.19011261, 17.6924588 , ..., 17.51666431,
       18.77812427, 17.84593445])

In [33]:
score_mean = round(rmse(y_train, y_pred_mean), 2)
score_mean

0.46

In [34]:
### Question 4
"""
* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0. 
* Try different values of `r` from this list: `[0, 0.01, 0.1, 1, 5, 10, 100]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

Options:

- 0
- 0.01
- 1
- 10
- 100
"""


"\n* Now let's train a regularized linear regression.\n* For this question, fill the NAs with 0. \n* Try different values of `r` from this list: `[0, 0.01, 0.1, 1, 5, 10, 100]`.\n* Use RMSE to evaluate the model on the validation dataset.\n* Round the RMSE scores to 2 decimal digits.\n* Which `r` gives the best RMSE?\n\nIf there are multiple options, select the smallest `r`.\n\nOptions:\n\n- 0\n- 0.01\n- 1\n- 10\n- 100\n"

In [35]:
def train_linear_regression_reg(X, y, r = 0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r*np.eye(XTX.shape[0])
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)


    return w_full[0], w_full[1:]

In [36]:
for r in [0, 0.01, 0.1, 1, 5, 10, 100]:

    X_train = df_train.fillna(0).values
    w0, w = train_linear_regression_reg(X_train, y_train, r = r)
    
    X_val =  df_val.fillna(0).values
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    print(r, round(score,2))

0 0.52
0.01 0.52
0.1 0.52
1 0.52
5 0.52
10 0.52
100 0.52


In [37]:
### Question 5
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
scores = []

for seed in seeds:
    idx = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(idx)
    df_train = df_col.iloc[idx[:n_train]]
    df_val = df_col.iloc[idx[n_train: n_train + n_val]]
    df_test = df_col.iloc[idx[n_train+n_val:]]
    
    df_train = df_train.reset_index(drop = True)
    df_val = df_val.reset_index(drop = True)
    df_test = df_test.reset_index(drop = True)
    
    y_train = df_train.fuel_efficiency_mpg
    y_val = df_val.fuel_efficiency_mpg
    y_test = df_test.fuel_efficiency_mpg
    
    del df_train['fuel_efficiency_mpg']
    del df_val['fuel_efficiency_mpg']
    del df_test['fuel_efficiency_mpg']
    X_train = df_train.fillna(0).values
    w0, w = train_linear_regression(X_train, y_train)
    
    X_val =  df_val.fillna(0).values
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    scores.append(score)
    print(seed, round(score,2))

std_scores = np.std(scores)
print(f'The value of std: {round(std_scores, 3)}')
print(f'The value of std: {(std_scores)}')

0 0.52
1 0.52
2 0.52
3 0.52
4 0.51
5 0.53
6 0.53
7 0.51
8 0.51
9 0.51
The value of std: 0.007
The value of std: 0.006989446426357497


In [38]:
### Question 6
"""
* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`. 
* What's the RMSE on the test dataset?

Options:

- 0.15
- 0.515
- 5.15
- 51.5
"""

"\n* Split the dataset like previously, use seed 9.\n* Combine train and validation datasets.\n* Fill the missing values with 0 and train a model with `r=0.001`. \n* What's the RMSE on the test dataset?\n\nOptions:\n\n- 0.15\n- 0.515\n- 5.15\n- 51.5\n"

In [39]:
idx = np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)

In [40]:
df_col = data[cols]
df_train = df_col.iloc[idx[:n_train]]
df_val = df_col.iloc[idx[n_train: n_train + n_val]]
df_test = df_col.iloc[idx[n_train+n_val:]]

In [41]:
y_train = df_train.fuel_efficiency_mpg
y_val = df_val.fuel_efficiency_mpg
y_test = df_test.fuel_efficiency_mpg

del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

In [42]:
df_full_train = pd.concat([df_train, df_val])
df_full_train = df_full_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [43]:
X_full_train = df_full_train.fillna(0).values
X_full_train

array([[ 240.        ,  136.        , 4050.51228957, 2006.        ],
       [ 190.        ,  141.        , 3195.86694158, 2017.        ],
       [ 200.        ,  184.        , 3006.16436968, 2017.        ],
       ...,
       [ 180.        ,  154.        , 3346.96567067, 2018.        ],
       [ 210.        ,  152.        , 2500.17568746, 2020.        ],
       [ 260.        ,  174.        , 2702.25730066, 2011.        ]])

In [44]:
y_full_train = np.concatenate([y_train, y_val])
y_full_train

array([10.30469994, 13.47943378, 16.19065297, ..., 13.67814034,
       17.59323665, 16.06897494])

In [45]:
w0, w = train_linear_regression_reg(X_full_train, y_full_train, r = 0.001)

In [46]:
X_test = df_test.fillna(0).values
y_pred = w0 + X_test.dot(w)
score = rmse(y_test, y_pred)
score

0.5156261299202296